### 필수과제1
- rfm을 구한 데이터셋을 가지고 -> LTV 계산
- 이미 과제를 통해 기준을 정했으니 -> LTV 값도 기준 및 특정 구매 패턴의 월에 따라 다시 추정해 볼 수 있다.
- 1월~3월까지 구매를 가지고 -> LTV 3개월 예측 -> 실제 구매한 값이 -> 실제값 - 예측값에 대한 오차
    - 내가 LTV로 계산한 미래의 값과, 데이터로 구매한 고객의 금액이 차이가 얼마나 나는지!?

### LTV 예측

In [1]:
import pandas as pd
import os

os.chdir(r'..')

In [2]:
import numpy as np
from datetime import timedelta
from lifetimes import BetaGeoFitter, GammaGammaFitter
from lifetimes.utils import summary_data_from_transaction_data

In [3]:
df = pd.read_excel('Online Retail.xlsx')

In [4]:
df['TotalPrice'] = df['Quantity']*df['UnitPrice']
df['InvoiceDay'] = df['InvoiceDate'].dt.strftime('%Y-%m-%d')

In [5]:
df_list = df[['CustomerID','InvoiceDay','TotalPrice']]
df_list_cleaned = df_list.groupby(['CustomerID', 'InvoiceDay']).agg(
                    Quantity=('TotalPrice', 'sum')).reset_index()

In [6]:
df_list_cleaned

,CustomerID,InvoiceDay,Quantity
0,12346.0,2011-01-18,0.00
1,12347.0,2010-12-07,711.79
2,12347.0,2011-01-26,475.39
3,12347.0,2011-04-07,636.25
4,12347.0,2011-06-09,382.52
...,...,...,...
19291,18283.0,2011-11-30,223.61
19292,18283.0,2011-12-06,208.00
19293,18287.0,2011-05-22,765.28
19294,18287.0,2011-10-12,1001.32


In [7]:
df_list_cleaned = df_list_cleaned[df_list_cleaned['Quantity']>0]

In [8]:
customer_counts = df_list_cleaned['CustomerID'].value_counts()
customers_with_multiple_purchases = customer_counts[customer_counts >= 2].index
df_multiple_purchases = df_list_cleaned[df_list_cleaned['CustomerID'].isin(customers_with_multiple_purchases)]
df_multiple_purchases

,CustomerID,InvoiceDay,Quantity
1,12347.0,2010-12-07,711.79
2,12347.0,2011-01-26,475.39
3,12347.0,2011-04-07,636.25
4,12347.0,2011-06-09,382.52
5,12347.0,2011-08-02,584.91
...,...,...,...
19291,18283.0,2011-11-30,223.61
19292,18283.0,2011-12-06,208.00
19293,18287.0,2011-05-22,765.28
19294,18287.0,2011-10-12,1001.32


In [31]:
observation_date =  pd.to_datetime('2011-12-10')

summary =summary_data_from_transaction_data(
    transactions = df_multiple_purchases,
    customer_id_col = 'CustomerID',
    datetime_col = 'InvoiceDay',
    monetary_value_col = 'Quantity',
    observation_period_end = observation_date,
    freq= 'D'
    )

In [32]:
summary

,frequency,recency,T,monetary_value
CustomerID,,,,
12347.0,6.0,365.0,368.0,599.701667
12348.0,3.0,283.0,359.0,301.480000
12352.0,6.0,260.0,297.0,208.151667
12356.0,2.0,303.0,326.0,269.905000
12358.0,1.0,149.0,151.0,683.200000
...,...,...,...,...
18272.0,5.0,244.0,247.0,487.752000
18273.0,2.0,255.0,258.0,76.500000
18282.0,1.0,119.0,127.0,77.840000


In [33]:
bgf = BetaGeoFitter(penalizer_coef=0.01)
bgf.fit(summary['frequency'], summary['recency'], summary['T'])

<lifetimes.BetaGeoFitter: fitted with 2777 subjects, a: 0.01, alpha: 91.67, b: 0.11, r: 1.61>

In [34]:
ggf = GammaGammaFitter(penalizer_coef=0.01)
ggf.fit(summary['frequency'], summary['monetary_value'])

<lifetimes.GammaGammaFitter: fitted with 2777 subjects, p: 3.76, q: 0.34, v: 3.65>

In [35]:
summary['expected_purchases_3m']=bgf.conditional_expected_number_of_purchases_up_to_time(
    90, summary['frequency'], summary['recency'], summary['T'])

In [36]:
summary['expected_avg_value']=ggf.conditional_expected_average_profit(summary['frequency'],summary['monetary_value'])

In [37]:
summary['expected_LTV_3m'] = summary['expected_purchases_3m'] * summary['expected_avg_value']

In [38]:
summary

,frequency,recency,T,monetary_value,expected_purchases_3m,expected_avg_value,expected_LTV_3m
CustomerID,,,,,,,
12347.0,6.0,365.0,368.0,599.701667,1.486781,618.529607,919.617833
12348.0,3.0,283.0,359.0,301.480000,0.913849,321.645659,293.935566
12352.0,6.0,260.0,297.0,208.151667,1.756188,215.095933,377.748904
12356.0,2.0,303.0,326.0,269.905000,0.771860,298.070144,230.068497
12358.0,1.0,149.0,151.0,683.200000,0.920379,834.302992,767.875246
...,...,...,...,...,...,...,...
18272.0,5.0,244.0,247.0,487.752000,1.751755,506.381358,887.056111
18273.0,2.0,255.0,258.0,76.500000,0.922648,85.917424,79.271533
18282.0,1.0,119.0,127.0,77.840000,1.017521,98.984253,100.718512


### RFM LTV 계산

In [52]:
df_rfm = pd.read_csv('Online Retail_RFM.csv', index_col=0)

In [ ]:
df_rfm_1 = df_rfm[df_rfm['BaseMonth']=='2011-01'][['CustomerID','Monetary','BaseMonth']]
df_rfm_2 = df_rfm[df_rfm['BaseMonth']=='2011-02'][['CustomerID','Monetary','BaseMonth']]
df_rfm_3 = df_rfm[df_rfm['BaseMonth']=='2011-03'][['CustomerID','Monetary','BaseMonth']]

df_rfm_agg = pd.concat([df_rfm_1, df_rfm_2, df_rfm_3], ignore_index=True)
df_rfm_agg_sum = df_rfm_agg.groupby('CustomerID')['Monetary'].sum()
df_rfm_agg_sum

CustomerID
12346.0      0.00
12347.0    475.39
12348.0    227.44
12350.0    334.40
12352.0    601.18
            ...  
18270.0    111.95
18273.0     51.00
18277.0    -12.75
18280.0    180.60
18283.0    317.90
Name: Monetary, Length: 1812, dtype: float64

In [92]:
merged_summary = pd.merge(summary, df_rfm_agg_sum, on='CustomerID', how='left')
merged_summary = merged_summary[merged_summary['Monetary']>0]
merged_summary

,frequency,recency,T,monetary_value,expected_purchases_3m,expected_avg_value,expected_LTV_3m,Monetary
CustomerID,,,,,,,,
12347.0,6.0,365.0,368.0,599.701667,1.486781,618.529607,919.617833,475.39
12348.0,3.0,283.0,359.0,301.480000,0.913849,321.645659,293.935566,227.44
12352.0,6.0,260.0,297.0,208.151667,1.756188,215.095933,377.748904,601.18
12356.0,2.0,303.0,326.0,269.905000,0.771860,298.070144,230.068497,2271.62
12359.0,3.0,274.0,332.0,1941.693333,0.972946,2064.537372,2008.683232,2386.41
...,...,...,...,...,...,...,...,...
18257.0,6.0,257.0,297.0,332.371667,1.755925,343.086051,602.433212,340.15
18260.0,5.0,186.0,359.0,482.500000,1.277845,500.936910,640.119888,1087.57
18270.0,1.0,228.0,267.0,171.200000,0.614136,212.386786,130.434270,111.95


### LTV 값 비교

In [93]:
merged_summary['Residual'] = merged_summary['Monetary']-merged_summary['expected_LTV_3m']
merged_summary['Absolute_Error'] = np.abs(merged_summary['Residual'])
merged_summary['Squared_Error'] = merged_summary['Residual']**2

In [95]:
merged_summary

,frequency,recency,T,monetary_value,expected_purchases_3m,expected_avg_value,expected_LTV_3m,Monetary,Residual,Absolute_Error,Squared_Error
CustomerID,,,,,,,,,,,
12347.0,6.0,365.0,368.0,599.701667,1.486781,618.529607,919.617833,475.39,-444.227833,444.227833,1.973384e+05
12348.0,3.0,283.0,359.0,301.480000,0.913849,321.645659,293.935566,227.44,-66.495566,66.495566,4.421660e+03
12352.0,6.0,260.0,297.0,208.151667,1.756188,215.095933,377.748904,601.18,223.431096,223.431096,4.992145e+04
12356.0,2.0,303.0,326.0,269.905000,0.771860,298.070144,230.068497,2271.62,2041.551503,2041.551503,4.167933e+06
12359.0,3.0,274.0,332.0,1941.693333,0.972946,2064.537372,2008.683232,2386.41,377.726768,377.726768,1.426775e+05
...,...,...,...,...,...,...,...,...,...,...,...
18257.0,6.0,257.0,297.0,332.371667,1.755925,343.086051,602.433212,340.15,-262.283212,262.283212,6.879248e+04
18260.0,5.0,186.0,359.0,482.500000,1.277845,500.936910,640.119888,1087.57,447.450112,447.450112,2.002116e+05
18270.0,1.0,228.0,267.0,171.200000,0.614136,212.386786,130.434270,111.95,-18.484270,18.484270,3.416682e+02


In [104]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

X = pd.DataFrame(merged_summary['Monetary'])
y = merged_summary['expected_LTV_3m']

X_train, X_test, y_train, y_test = train_test_split(X, y , test_size=0.2, random_state=1)

lr_rg = LinearRegression()
lr_rg.fit(X_train, y_train)
y_pred = lr_rg.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print('MAE : ', mae)
print('MSE : ', mse)
print('R2 : ', r2)

MAE :  601.5615442340918
MSE :  5464457.32807302
R2 :  0.7818412702923949


In [113]:
import plotly.express as px

fig = px.scatter(data_frame=merged_summary, x='Monetary', y='expected_LTV_3m',
                 labels={'Monetary':'Actual LTV', 'expected_LTV_3m':'Expected LTV'},
                 title='LTV Comparison', trendline='ols')

fig.show()